In [ ]:
import nltk
from nltk import pos_tag, word_tokenize
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [ ]:
from collections import Counter, defaultdict
from more_itertools import unique_everseen
import re

In [ ]:
'''WordNet is a semantically-oriented dictionary of English, similar to a traditional thesaurus but 
with a richer structure. NLTK includes the English WordNet, with 155,287 words and 117,659 synonym sets. 
We'll begin by looking at synonyms and how they are accessed in WordNet.'''

print(wn.synsets('car'))


In [ ]:
for synset in wn.synsets('car'):
    print(synset.lemma_names())

In [ ]:
wn.synset('car.n.01').definition()

In [ ]:
wn.synset('car.n.01').examples()

In [ ]:
'''Navigate between concepts with hyponyms'''
car = wn.synset('car.n.01')
types_of_cars = car.hyponyms()
print(sorted(lemma.name() for synset in types_of_cars for lemma in synset.lemmas()))

In [ ]:
'''You can also navigate up the hierarchy by visiting hypernyms. 
Some words have multiple paths, because they can be classified in more than one way. 
There are two paths between car.n.01 and entity.n.01 because 
wheeled_vehicle.n.01 can be classified as both a vehicle and a container.'''

print(car.hypernyms())
paths = car.hypernym_paths()
len(paths)


In [ ]:
print([synset.name() for synset in paths[0]])
print('---------------------------------------')
print([synset.name() for synset in paths[1]])

In [ ]:
'''We can get the most general hypernyms (or root hypernyms) of a synset as follows'''
car.root_hypernyms()

In [ ]:
'''More Lexical Relations'''
wn.synset('tree.n.01').part_meronyms()

In [ ]:
#Meronyms
wn.synset('tree.n.01').substance_meronyms()

In [ ]:
#Holonyms
wn.synset('tree.n.01').member_holonyms()

In [ ]:
for synset in wn.synsets('mint', wn.NOUN):
    print(synset.name() + ':', synset.definition())

In [ ]:
print(wn.synset('mint.n.04').part_holonyms())
print(wn.synset('mint.n.04').substance_holonyms())

In [ ]:
#Entailment
print(wn.synset('walk.v.01').entailments())
#print(wn.synset('eat.v.01').entailments())

In [ ]:
#Antonyms
print(wn.lemma('supply.n.02.supply').antonyms())
print(wn.lemma('rush.v.01.rush').antonyms())
print(wn.lemma('horizontal.a.01.horizontal').antonyms())
print(wn.lemma('staccato.r.01.staccato').antonyms())


In [ ]:
'''Semantic Similarity'''
right = wn.synset('right_whale.n.01')
orca = wn.synset('orca.n.01')
minke = wn.synset('minke_whale.n.01')
tortoise = wn.synset('tortoise.n.01')
novel = wn.synset('novel.n.01')
print("Minke hypernym:")
print(right.lowest_common_hypernyms(minke))
print("Orca: hypernym: ")
print(right.lowest_common_hypernyms(orca))
print("Tortoise hypernym: ")
print(right.lowest_common_hypernyms(tortoise))
print("Novel hypernym: ")
print(right.lowest_common_hypernyms(novel))

In [ ]:
#synset depth
wn.synset('baleen_whale.n.01').min_depth()

#wn.synset('whale.n.02').min_depth()

#wn.synset('vertebrate.n.01').min_depth()

#wn.synset('entity.n.01').min_depth()


In [ ]:
'''Similarity Scores'''
right.path_similarity(minke)

# right.path_similarity(orca)

# right.path_similarity(tortoise)

# right.path_similarity(novel)

In [ ]:
'''Example: Using WordNet to find organisms in biology texts'''

senses = (wn.synsets('plant', pos="n"))
print(senses)

In [ ]:
s1 = senses[0]
s2 = senses[1]
s3 = senses[2]
s4 = senses[3]
d1 = (s1.definition()) 
print("Def 1: " + str(d1))
d2 = (s2.definition())
print("Def 2: " + str(d2))
d3 = (s3.definition())
print("Def 3: " + str(d3))
d4 = (s4.definition()) 
print("Def 4: " + str(d4))

In [ ]:
#Load text
def readMe(file):
    raw_text = open(file, 'r')
    raw_text = raw_text.read()
    return raw_text

text = readMe("/Users/hclent/Desktop/NLPtutorial/texts/18952863_4.txt")
#print(fcorpus)


In [ ]:
def formatCorpus(corpus): #as string
    corpus = corpus.replace("_", "underscore") #underscores will make problems for buildDict and are unimportant for my NER
    #make untagged corpus
    untagged_corpus = corpus.lower()
    #untagged_corpus = untagged_corpus.split() #tokenize
    untagged_corpus = nltk.word_tokenize(untagged_corpus)
    stopwords = nltk.corpus.stopwords.words('english') #delete stopwords from untagged
    untagged_corpus = [w for w in untagged_corpus if w.lower() not in stopwords]


    return untagged_corpus

untagged_corpus = formatCorpus(text)
#print(untagged_corpus)


In [ ]:
#Use WordNet to find organisms
def wordNetNER(document):
    plant_sns = (wn.synsets('plant', pos="n"))
    plant = plant_sns[1] #(botany) a living organism lacking the power of locomotion #hardcoded

    wordnet_names = []
    
    for word in document:

        mySynsets = wn.synsets(word, pos="n") #look at nouns


        i = 0
        for i in range(0, 3):
            try:
                given_word = mySynsets[i] #tries first 3 synsets
                definition = (given_word.definition())
                p1 = re.compile('plant(s?)\s')
                p2 = re.compile('organism(s?)\s')
                p3 = re.compile('animal(s?)\s')
                match1 = p1.search(definition)
                match2 = p2.search(definition)
                match3 = p3.search(definition)

                if match1 or match2 or match3:  #if word has "plants" or "animals" in the def, check how similar"
                    similarity_score = (given_word.path_similarity(plant)) #check similarity score
                    if similarity_score >= 0.2:
                        #print(similarity_score)
                        #print ("The words: "+(str(given_word)) + "  has a sim score of:  " +str(similarity_score))
                        wordnet_names.append(word)
                i += 1
            except IndexError:
                pass

    wordnet_ner = (list(unique_everseen(wordnet_names)))
    return wordnet_ner

wordnet_names = wordNetNER(untagged_corpus)


In [ ]:
print("WORDNET: ")
print(wordnet_names)